In [15]:
#Grab all catchments from Tague and Grant (2004), perform recession extraction,
#analysis, de-correlation, and comparative analysis. 

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import scipy.stats as sp
import scipy.optimize as op
import statsmodels.api as sm
import ulmo
from pandas.stats.api import ols
from datetime import datetime, date, timedelta
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.pyplot import cm 
from pylab import rcParams
import seaborn as sns
rcParams['figure.figsize'] = 10, 8

#functions written by Paul Inkenbrandt
#need ulmo package, 'conda install -c IOOS ulmo'
def importusgssite(siteno, startdate, enddate):
    sitename = {};
    sitename = ulmo.usgs.nwis.get_site_data(siteno, service="iv", start =startdate, end = enddate);
    sitename = pd.DataFrame(sitename)
    sitename = pd.DataFrame(sitename['00060:00011']['values']);
    sitename['dates'] = pd.to_datetime(pd.Series(sitename['datetime']));
    sitename.set_index(['dates'],inplace=True);
#     sitename[siteno] = sitename['value'].astype(float);
#     sitename[str(siteno)+'qual'] = sitename['qualifiers'];
#     sitename = sitename.drop(['datetime','qualifiers','value'],axis=1);
     sitename = sitename.replace('-999999',np.NAN);
#     sitename = sitename.dropna();
    #sitename['mon']=sitename.index.month
    return sitename;

def getusgssiteinfo(siteno):
    siteinfo = ulmo.usgs.nwis.get_site_data(siteno, service="daily", period="all")
    siteinfo = pd.DataFrame(siteinfo['00060:00003']['site'])
    siteinfo['latitude'] = siteinfo.loc['latitude','location']
    siteinfo['longitude'] = siteinfo.loc['longitude','location']
    siteinfo['latitude'] = siteinfo['latitude'].astype(float)
    siteinfo['longitude'] = siteinfo['longitude'].astype(float)
    siteinfo = siteinfo.drop(['default_tz','dst_tz','srs','uses_dst','longitude'],axis=0)
    siteinfo = siteinfo.drop(['agency','timezone_info','location','state_code','network'],axis=1)
    return siteinfo
#
#def recessionExtract(df, lag, ante, start, alph, smoother, selectivity, minLen, option) :
#    df_smooth = pd.rolling_mean(df,smoother);
site = '11475560';
startdate = '2015-10-01'
enddate = '2016-02-09'
df = importusgssite(site, startdate, enddate);
#del d[list(d)[1]]



INFO:ulmo.usgs.nwis.core:processing data from request: http://nwis.waterservices.usgs.gov/nwis/iv/?endDT=2016-02-09T00%3A00%3A00&startDT=2015-10-01T00%3A00%3A00&site=11475560&format=waterml


In [16]:
df

,datetime,qualifiers,value
dates,,,
2015-10-01 07:00:00,2015-10-01T00:00:00-07:00,P,0.46
2015-10-01 07:15:00,2015-10-01T00:15:00-07:00,P,0.46
2015-10-01 07:30:00,2015-10-01T00:30:00-07:00,P,0.46
2015-10-01 07:45:00,2015-10-01T00:45:00-07:00,P,0.46
2015-10-01 08:00:00,2015-10-01T01:00:00-07:00,P,0.46
2015-10-01 08:15:00,2015-10-01T01:15:00-07:00,P,0.46
2015-10-01 08:30:00,2015-10-01T01:30:00-07:00,P,0.46
2015-10-01 08:45:00,2015-10-01T01:45:00-07:00,P,0.46
2015-10-01 09:00:00,2015-10-01T02:00:00-07:00,P,0.46
